<a href="https://colab.research.google.com/github/hamidsadeghi68/face-clustering/blob/main/face_clustering_arcface_facecup_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**install** insighface package for google colab

In [ ]:
!pip install -U insightface

import torch
if torch.cuda.is_available():
  print('gpu is available')
  !pip install onnxruntime-gpu==1.9.0   # to use GPU
else:
  print('gpu is not available')
  !pip install onnxruntime

     |████████████████████████████████| 431 kB 4.1 MB/s 
     |████████████████████████████████| 12.7 MB 44.8 MB/s 
     |████████████████████████████████| 631 kB 42.6 MB/s 
  Created wheel for insightface: filename=insightface-0.5-cp37-cp37m-linux_x86_64.whl size=975559 sha256=c0a1f70fd153337eff788911a1677a458a033a0b0b58fdd128b80085999c4965
  Stored in directory: /root/.cache/pip/wheels/10/41/10/d7c3a6acc5896f7e5ebdeb0456579bc6e1dce43bf4920f23a4
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=96c8922ca336a47d831fc722c79b110eeee74e096782d1ccc4d34d589c5d4a2b
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built insightface imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
gpu is available
     |████████████████████████████████| 95.6 MB 104 kB/s 


**download [facecup](https://facecup.ir/) sample dataset**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!mkdir datasets
# # https://drive.google.com/file/d/1hEyOu5u8ZwdNfBfZL7SeloQTpz3Ti79u/view?usp=sharing
# !gdown --id 1hEyOu5u8ZwdNfBfZL7SeloQTpz3Ti79u -O "./datasets/samples for participants.rar"
# !unrar x -Y "./datasets/samples for participants.rar" "./datasets/samples for participants/"

!unrar x -Y '/gdrive/My Drive/datasets/samples for participants.rar' "./datasets/samples for participants/"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /gdrive/My Drive/datasets/samples for participants.rar

Creating    ./datasets/samples for participants                       OK
Creating    ./datasets/samples for participants/images                OK
Extracting  ./datasets/samples for participants/images/A0101.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0102.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0103.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0104.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0105.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0106.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0107.jpg           0%  OK 
Extracting  ./datasets/samples for participants/images/A0108.jpg     

**prepare insightface module**

Here I used default model. You can select the best Face Detection and Face Recognition models (trade-off between accuracy and computational cost) on model-zoo page

more accurate <===================================================> faster

ResNet-100 (R100) - ResNet-50 (R50) - ResNet-34 (R34) - ResNet-18 (R18) - MobileNet

In [ ]:
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

app = FaceAnalysis()
app.prepare(ctx_id=0, det_thresh=0.5)

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:16<00:00, 17128.86KB/s]
/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:353: UserWarning: Deprecation warning. This ORT build has ['CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. The next release (ORT 1.10) will require explicitly setting the providers parameter (as opposed to the current behavior of providers getting set/registered by default based on the build flags) when instantiating InferenceSession.For example, onnxruntime.InferenceSession(..., providers=["CUDAExecutionProvider"], ...)
  "based on the build flags) when instantiating InferenceSession."


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'arena_extend_strategy': 'kNextPowerOfTwo', 'gpu_external_empty_cache': '0', 'gpu_external_free': '0', 'cudnn_conv_use_max_workspace': '0', 'gpu_mem_limit': '18446744073709551615', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_alloc': '0', 'device_id': '0'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'do_copy_in_default_stream': '1', 'arena_extend_strategy': 'kNextPowerOfTwo', 'gpu_external_empty_cache': '0', 'gpu_external_free': '0', 'cudnn_conv_use_max_workspace': '0', 'gpu_mem_limit': '18446744073709551615', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'gpu_external_alloc': '0', 'device_id': '0'}}
find model: /ro

**Important notes about run time:**


> a) Remove unnecessary processes, such as landmark detection and gender recognition, to reduce run time.
in [face_analysis.py](https://github.com/deepinsight/insightface/blob/8137469a08a00ddf9a4de4f15f5e98b2869ced21/python-package/insightface/app/face_analysis.py#L73) (/usr/local/lib/python3.7/dist-packages/insightface/app/face_analysis.py) see: "taskname, model in self.models.items():"
*   'landmark_3d_68'
*   'landmark_2d_106'
*   'genderage'
*   'recognition'
*   'detection'

> b) Feature (embedding) extraction can be perfomed only for the best face ([facecup](https://facecup.ir/) images are single face).



> c) Try to use batch processing (feature extraction) to reduce overall run time.








**feature extraction**

In [ ]:
import os
import cv2
import numpy as np

dataset_path = './datasets/samples for participants/images'
files = os.listdir(dataset_path)

accepted_files = []
features = []
labels = []
reject_labels = []
counter = 1
for file in files:
  if not file.endswith('.jpg'):
    continue
  print('prosessing image %d: %s'%(counter,file))
    
  img = cv2.imread(os.path.join(dataset_path, file))
  faces = app.get(img)
  if len(faces) > 0:
    features.append(faces[0].normed_embedding)
    accepted_files.append(file)
    labels.append(int(file[1:3]))
  else:
    reject_labels.append(int(file[1:3]))
  
  counter+=1

labels += reject_labels # add images without face detection
print('number of images without face: %d' % len(reject_labels))

prosessing image 1: A0118.jpg
prosessing image 2: A0907.jpg
prosessing image 3: A2403.jpg
prosessing image 4: A0257.jpg
prosessing image 5: A0517.jpg
prosessing image 6: A0528.jpg
prosessing image 7: A2513.jpg
prosessing image 8: A1904.jpg
prosessing image 9: A0103.jpg
prosessing image 10: A1534.jpg
prosessing image 11: A1530.jpg
prosessing image 12: A1008.jpg
prosessing image 13: A2516.jpg
prosessing image 14: A1444.jpg
prosessing image 15: A0333.jpg
prosessing image 16: A1921.jpg
prosessing image 17: A1915.jpg
prosessing image 18: A1803.jpg
prosessing image 19: A1443.jpg
prosessing image 20: A0802.jpg
prosessing image 21: A0138.jpg
prosessing image 22: A0602.jpg
prosessing image 23: A1114.jpg
prosessing image 24: A2015.jpg
prosessing image 25: A0933.jpg
prosessing image 26: A0806.jpg
prosessing image 27: A0104.jpg
prosessing image 28: A2530.jpg
prosessing image 29: A0936.jpg
prosessing image 30: A0141.jpg
prosessing image 31: A0701.jpg
prosessing image 32: A0238.jpg
prosessing image 

**Evaluation**

In [ ]:
# download evaluation.py from my github page to use in colab machine
!wget https://raw.githubusercontent.com/hamidsadeghi68/face-clustering/main/evaluation.py

--2021-12-11 07:48:54--  https://raw.githubusercontent.com/hamidsadeghi68/face-clustering/main/evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 674 [text/plain]
Saving to: ‘evaluation.py’

evaluation.py       100%[===================>]     674  --.-KB/s    in 0s      

2021-12-11 07:48:54 (32.6 MB/s) - ‘evaluation.py’ saved [674/674]



In [ ]:
from evaluation import evaluate_clustering

**Clustering**

K-means clustering

In [ ]:
from sklearn import cluster
k = 40 # set the number of clusters
y_pred = cluster.KMeans(n_clusters=k, random_state=0).fit_predict(features)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 63
Rand Index: 0.972927, NMI: 0.875660, F-measure: 0.666055


DBSCAN clustering

In [ ]:
y_pred = cluster.DBSCAN(eps=1.14, min_samples=2).fit_predict(features)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 57
Rand Index: 0.993958, NMI: 0.946196, F-measure: 0.938772


Agglomerative clustering (using feature vector directly)

In [ ]:
y_pred = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=1.14, linkage='single').fit_predict(features)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 83
Rand Index: 0.994733, NMI: 0.959152, F-measure: 0.946191


Agglomerative clustering (using distance matrix)

In [ ]:
!wget https://raw.githubusercontent.com/hamidsadeghi68/face-clustering/main/matching.py
from matching import match

--2021-12-11 09:41:30--  https://raw.githubusercontent.com/hamidsadeghi68/face-clustering/main/matching.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548 [text/plain]
Saving to: ‘matching.py’

matching.py         100%[===================>]     548  --.-KB/s    in 0s      

2021-12-11 09:41:31 (26.4 MB/s) - ‘matching.py’ saved [548/548]



In [ ]:
# re-implement this part of code to reduce run-time:
n = len(features)
distance_matrix = np.empty([n, n])
for i, feature in enumerate(features):
  for j in range(i+1, n): # distance[i, j] == distance [j, i]
    distance_matrix[i, j] = match(feature, features[j])
    # distance_matrix[i, j] = match(feature, features[j], metric='Cosine') # precomputed --> a distance matrix is needed

In [ ]:
# affinity='precomputed'
y_pred = cluster.AgglomerativeClustering(n_clusters=None, distance_threshold=1.14, linkage='single', affinity='precomputed').fit_predict(distance_matrix)
y_pred_compelet = list(y_pred) + list(range(max(y_pred)+1,max(y_pred)+len(reject_labels)+1))
print('num of clusters: %d' % (max(y_pred_compelet)+1))
evaluate_clustering(labels, y_pred_compelet)

num of clusters: 83
Rand Index: 0.994733, NMI: 0.959152, F-measure: 0.946191
